In [1]:
import pandas as pd
import ast
import warnings
warnings.filterwarnings("ignore")
df_movies = pd.read_parquet("../etl/dataset/movies_dataset.parquet")

- Aca vuelvo a desanidar los datos del df para poder hacer un analisis de cada una de las columnas con mayor detalle, por que anteriormente habia hecho un etl de peliculas sin embargo, ese etl estaba mas centrado en responder a las primeras 4 consignas del proyecto y me enfoque mas en quitar y quitar columnas para hacerme el trabajo mas facil

In [2]:

#convierto a listas las filas que estaban como strings asi puedo desanidarlas despues recorriendo cada uno de sus elementos
df_movies['genres'] = df_movies['genres'].apply(lambda x: ast.literal_eval(x) if isinstance(x,str) else None)
df_movies['belongs_to_collection'] = df_movies['belongs_to_collection'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else None)
df_movies['production_companies'] = df_movies['production_companies'].apply(lambda x: ast.literal_eval(x) if isinstance(x,str) else None)
df_movies['production_countries'] = df_movies['production_countries'].apply(lambda x: ast.literal_eval(x) if isinstance(x,str) else None)
df_movies['spoken_languages'] = df_movies['spoken_languages'].apply(lambda x: ast.literal_eval(x) if isinstance(x,str) else None)
# desanido la columna de generos, de todas creo que name_genres es una de las mas valiosas para analizar
df_movies['id_genres'] = df_movies['genres'].apply(lambda x: [d['id'] for d in x])
df_movies['name_genres'] = df_movies['genres'].apply(lambda x: [d['name'] for d in x])
# esta es la columna que para mi tiene menos relevancia, no hay datos que puedan ser de utilidad a analizar, ni siquiera name_collection
df_movies['id_collection'] = df_movies['belongs_to_collection'].apply(lambda x: x['id'] if isinstance(x,dict) else None)
#################### si llegara name_collection, no quiero que quede mas texto que un 0, ya que podria usar para identificar facilmente a una saga
df_movies['name_collection'] = df_movies['belongs_to_collection'].apply(lambda x: x['name'] if isinstance(x,dict) else "0")
df_movies['poster_collection'] = df_movies['belongs_to_collection'].apply(lambda x: x['poster_path'] if isinstance(x,dict) else None)
df_movies['backdrop_collection'] = df_movies['belongs_to_collection'].apply(lambda x: x['backdrop_path'] if isinstance(x,dict) else None)
# En terminos de generos usualmente las compania suelen ser muy variadas, por lo que no veo importancia en estas columnas
df_movies['id_prod_companies'] = df_movies['production_companies'].apply(lambda x: [d['id'] for d in x] if isinstance(x, list) else [])
df_movies['name_prod_companies'] = df_movies['production_companies'].apply(lambda x: [d['name'] for d in x] if isinstance(x, list) else [])
# Lo mismo que ocurre en companies, todos los paises comparten generos variados por lo que no lo veo relevante
df_movies['iso_prod_countries'] = df_movies['production_countries'].apply(lambda x: [d['iso_3166_1'] for d in x] if isinstance(x, list) else [])
df_movies['name_prod_countries'] = df_movies['production_countries'].apply(lambda x: [d['name'] for d in x] if isinstance(x, list) else [])
# sin uso o caracteristicas que definan a una pelicula
df_movies['iso_spoken_languages'] = df_movies['spoken_languages'].apply(lambda x: [d['iso_639_1'] for d in x] if isinstance(x, list) else [])
df_movies['name_spoken_languages'] = df_movies['spoken_languages'].apply(lambda x: [d['name'] for d in x] if isinstance(x, list) else [])

In [3]:
# ya que estan las columnas desanidadas, las voy a eliminar
df_movies = df_movies.drop(['belongs_to_collection','genres','production_countries','spoken_languages','production_companies'], axis=1)
df_movies['id'] = pd.to_numeric(df_movies['id'], errors='coerce')
df_movies['id_collection'] = pd.to_numeric(df_movies['id_collection'], errors='coerce')
df_movies['budget'] = pd.to_numeric(df_movies['budget'], errors='coerce')


In [4]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   budget                 45463 non-null  float64
 2   homepage               7782 non-null   object 
 3   id                     45463 non-null  float64
 4   imdb_id                45449 non-null  object 
 5   original_language      45455 non-null  object 
 6   original_title         45466 non-null  object 
 7   overview               44512 non-null  object 
 8   popularity             45460 non-null  float64
 9   poster_path            45080 non-null  object 
 10  release_date           45379 non-null  object 
 11  revenue                45460 non-null  float64
 12  runtime                45203 non-null  float64
 13  status                 45379 non-null  object 
 14  tagline                20412 non-null  object 
 15  ti

In [5]:
# Aca lo que voy a hacer es rellenar las filas que aparecen vacias en title con las filas que si contengan datos en la columna original_title
df_movies["title"].fillna(df_movies["original_title"], inplace=True)

In [6]:
# Filtro los valores de Adult, ya que esta columna sera eliminada y me quedo con todas las peliculas que no tengan clasificacion 'X'
df_movies = df_movies[df_movies['adult']=='False']
# Elimino las columnas que mencionan en la consigna del proyecto
df_movies = df_movies.drop(['adult','homepage','imdb_id','original_title','poster_path','video'],axis=1)

Mi idea es tener claros los datos, por lo que quiero agruparlos por sus tipos  
1 - numericos :  
a.discretos  
b.continuos  
  

2 - categoricos :  
a.nominales  
b.ordinales  
c.binarios

- El unico ID que vamos a utilizar es el de pelicula para referenciarlo a la base de datos
- Los ISO tampoco son necesarios, voy a eliminarlos para evitar mucho tiempo de graficacion
- Datos como paginas web o 'paths', tambien seran eliminados
- Overview si bien no es un dato categorico lo voy a dejar para analizarlo con una nube de palabras y ver si encuentro algo de valor en el

In [7]:
df_movies = df_movies.drop(['id_collection','id_genres','id_prod_companies','iso_spoken_languages','iso_prod_countries','tagline','poster_collection','backdrop_collection', 'name_spoken_languages','name_prod_companies','name_prod_countries', 'original_language'],axis=1)

- Voy a pasar las fechas de lanzamiento en años, quiero ver mas adelante si me sirve este tipo de dato cuando realice el EDA

###
###

*lo siguiente que voy a hacer es en caso de utilizar name_collection, que no tome los str 'collection', por que este se repite en varias peliculas que si pertenecen a una saga o secuela de peliculas*

In [8]:
df_movies['name_collection'].value_counts()

name_collection
0                                40963
The Bowery Boys                     29
Totò Collection                     27
James Bond Collection               26
Zatôichi: The Blind Swordsman       26
                                 ...  
Сказки Чуковского                    1
Adventures of a ...                  1
The Cutlass Collection               1
Aniskin Collection                   1
Зимняя Вишня (трилогия)              1
Name: count, Length: 1696, dtype: int64

In [9]:
df_movies['name_collection'] = df_movies['name_collection'].str.replace('Collection', '')

In [10]:
df_movies['name_collection'].value_counts()

name_collection
0                                40963
The Bowery Boys                     29
Totò                                27
James Bond                          26
Zatôichi: The Blind Swordsman       26
                                 ...  
Сказки Чуковского                    1
Adventures of a ...                  1
The Cutlass                          1
Aniskin                              1
Зимняя Вишня (трилогия)              1
Name: count, Length: 1696, dtype: int64

In [11]:
df_movies['release_date'] = pd.to_datetime(df_movies['release_date'])
df_movies['release_year'] = df_movies['release_date'].dt.year
df_movies = df_movies.drop('release_date',axis=1)
# Le doy un poco de orden a la aparicion de las columnas
df_organizado = df_movies[['id','title','name_genres','budget','revenue','runtime','popularity','name_collection','vote_count','vote_average','status','overview','release_year']]

In [12]:
df_organizado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45454 entries, 0 to 45465
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               45454 non-null  float64
 1   title            45454 non-null  object 
 2   name_genres      45454 non-null  object 
 3   budget           45454 non-null  float64
 4   revenue          45451 non-null  float64
 5   runtime          45194 non-null  float64
 6   popularity       45451 non-null  float64
 7   name_collection  45454 non-null  object 
 8   vote_count       45451 non-null  float64
 9   vote_average     45451 non-null  float64
 10  status           45370 non-null  object 
 11  overview         44500 non-null  object 
 12  release_year     45368 non-null  float64
dtypes: float64(8), object(5)
memory usage: 4.9+ MB


In [13]:
df_organizado.to_parquet("dataset/movies.parquet")